In [ ]:
import os
import numpy as np

from IPython.display import clear_output 

In [ ]:
!wget 'https://github.com/ThanhChinhBK/vietnews/archive/master.zip'
!unzip 'master.zip'

clear_output()

In [ ]:
# Install the vncorenlp python wrapper
!pip install vncorenlp

# Download VnCoreNLP-1.1.1.jar & its word segmentation component (i.e. RDRSegmenter) 
!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/ 
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/
!pip install datasets==1.0.2

clear_output()

In [ ]:
import glob
import pandas as pd
import concurrent.futures

from datasets import *

In [ ]:
from vncorenlp import VnCoreNLP

rdrsegmenter = VnCoreNLP("./vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx2g') 

In [ ]:
def listPaths(path):
  pathfiles = list()
  for pathfile in glob.glob(path):
    pathfiles.append(pathfile)
  return pathfiles

train_paths = listPaths('/content/vietnews-master/data/train_tokenized/*')
val_paths = listPaths('/content/vietnews-master/data/val_tokenized/*')
test_paths = listPaths('/content/vietnews-master/data/test_tokenized/*')

In [ ]:
def read_content(pathfile):
  """
  Input: Path of txt file
  Output: A dictionary has keys 'original' and 'summary'
  """
  with open(pathfile) as f:
    rows  = f.readlines()
    original = ' '.join(''.join(rows[4:]).split('\n'))
    summary = ' '.join(rows[2].split('\n'))
          
  return {'file' : pathfile,
          'original': original, 
          'summary': summary}

In [ ]:
read_content(train_paths[0])

{'file': '/content/vietnews-master/data/train_tokenized/033270.txt.seg',
 'original': 'Theo một hãng tin của Syria , cuối tuần qua , quân Chính_phủ Syria với sự hỗ_trợ của các đồng_minh đã giành lại được nhiều khu_vực lãnh_thổ từ lực_lượng khủng_bố IS ở Raqqa và Deir al - Zor . Quân_đội đã tái_chiếm được một mỏ dầu ở khu_vực Sabkha trong quá_trình oanh kích khu_vực . Đây là một bước_tiến hiếm_hoi đối_với các lực_lượng của Damascus trong khu_vực này , gần lãnh_thổ được kiểm_soát bởi Lực_lượng Dân_chủ Syria , liên_minh người Kurd đang hoạt_động chống lại IS ở Syria . Đợt oanh kích lần này cũng đưa lực_lượng chính_phủ Syria tới gần hơn với tỉnh Deir al - Zor , một “ thành_trì ” khác mà IS đang chiếm_đóng . Những sự_cố hồi tháng trước giữa quân_đội Syria và SDF đã làm gia_tăng căng_thẳng giữa Washington và Damascus cùng các đồng_minh . Ở khu_vực phía Bắc_Syria , một trong những chiến_trường phức_tạp nhất tại quốc_gia Trung_Đông này , các phiến quân Syria do Thổ_Nhĩ_Kỳ hậu_thuẫn đang chiến_

In [ ]:
def get_dataframe(pathfiles):
  with concurrent.futures.ProcessPoolExecutor() as executor:
    data = executor.map(read_content, pathfiles)
  
  # Make blank dataframe
  data_df = list()
  for d in data:
    data_df.append(d)
  data_df = pd.DataFrame(data_df)
  data_df.dropna(inplace=True)
  data_df = data_df.sample(frac=1).reset_index(drop=True)

  return data_df

In [ ]:
train_df = get_dataframe(train_paths)
val_df = get_dataframe(val_paths)
test_df = get_dataframe(test_paths)

In [ ]:
!pip install datasets==1.0.2
!pip install transformers==4.1.1

import datasets
import transformers

clear_output()

In [ ]:
train_df

,file,original,summary
0,/content/vietnews-master/data/train_tokenized/...,"Ngày 22-12 , luật_sư Phan_Trung_Hoài - người b...","Theo luật_sư Phan_Trung_Hoài , ông Đinh_La_Thă..."
1,/content/vietnews-master/data/train_tokenized/...,"Ngoại_trưởng Mỹ John_Kerry tại Ấn_Độ . "" Đây l...",Ngoại_trưởng Mỹ John_Kerry hôm_nay kêu_gọi Tru...
2,/content/vietnews-master/data/train_tokenized/...,Chưa ai dám nói tất_cả những người đi trên chu...,Tín_hiệu của sự sống vẫn là con_số 0 trong suố...
3,/content/vietnews-master/data/train_tokenized/...,Đối_tượng đề_cập trong bài viết này là Diệp_Vă...,"Học_hành dang_dở , sớm va_chạm xã_hội , nhưng ..."
4,/content/vietnews-master/data/train_tokenized/...,Thậm_chí một_số doanh_nghiệp viễn_thông lớn vẫ...,"Dù được trả lương qua thẻ , nhiều người vẫn rú..."
...,...,...,...
995,/content/vietnews-master/data/train_tokenized/...,"Phát_hiện nơi đây có mỏ nước nóng , công_ty Th...","Sau một đời chinh_chiến , đại_tá Tầm trở_về là..."
996,/content/vietnews-master/data/train_tokenized/...,“ Chúng_tôi nhận ra mong_muốn của các quốc_gia...,Mỹ đã lên_tiếng bác_bỏ quyết_định của Armenia ...
997,/content/vietnews-master/data/train_tokenized/...,"Tổng_thống Mỹ Donald_Trump . "" Chúng_tôi đã nó...",Tổng_thống Mỹ Donald_Trump ngày 2/8 cho_biết đ...
998,/content/vietnews-master/data/train_tokenized/...,"Theo Reuters , đây là một chiến_thắng mà phía ...","Ngày 2-12 ( giờ Mỹ ) , Thượng_viện Mỹ , do phe..."


In [ ]:
from transformers import RobertaTokenizerFast, AutoTokenizer

# phobert = AutoModel.from_pretrained("vinai/phobert-base")

# For transformers v4.x+: 
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)

Downloading:   0%|          | 0.00/557 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/895k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [ ]:
from sklearn.model_selection import train_test_split

train_data =  Dataset.from_pandas(train_df)
val_data =  Dataset.from_pandas(val_df)
test_data =  Dataset.from_pandas(test_df)

In [ ]:
train_data

Dataset(features: {'file': Value(dtype='string', id=None), 'original': Value(dtype='string', id=None), 'summary': Value(dtype='string', id=None)}, num_rows: 1000)

In [ ]:
tokenizer.decode(tokenizer('Tôi đi học đcm').input_ids)

'<s> Tôi đi học đcm </s>'

In [ ]:
batch_size = 16  # change to 16 for full training
encoder_max_length = 256
decoder_max_length = 64

def process_data_to_model_inputs(batch):                                                               
    # Tokenizer will automatically set [BOS] <text> [EOS]                                               
    inputs = tokenizer(batch["original"], padding="max_length", truncation=True, max_length=encoder_max_length)
    outputs = tokenizer(batch["summary"], padding="max_length", truncation=True, max_length=decoder_max_length)
                                                                                                        
    batch["input_ids"] = inputs.input_ids                                                               
    batch["attention_mask"] = inputs.attention_mask                                                     
    batch["decoder_input_ids"] = outputs.input_ids                                                      
    batch["labels"] = outputs.input_ids.copy()                                                          
    # mask loss for padding                                                                             
    batch["labels"] = [                                                                                 
        [-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]
    ]                     
    batch["decoder_attention_mask"] = outputs.attention_mask                                                                              
                                                                                                         
    return batch  

# only use 32 training examples for notebook - DELETE LINE FOR FULL TRAINING
# train_data = train_data.select(range(32))

train_data_batch = train_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["file", "original", "summary"],
)
train_data_batch.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)


# only use 16 training examples for notebook - DELETE LINE FOR FULL TRAINING
# val_data = val_data.select(range(16))

val_data_batch = val_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["file", "original", "summary"],
)
val_data_batch.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
) 

  0%|          | 0/63 [00:00<?, ?ba/s]

  0%|          | 0/32 [00:00<?, ?ba/s]

In [ ]:
from transformers import EncoderDecoderModel

# set encoder decoder tying to True
roberta_shared = EncoderDecoderModel.from_encoder_decoder_pretrained("vinai/phobert-base", "vinai/phobert-base", tie_encoder_decoder=True)

Downloading:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of RobertaForCausalLM were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['roberta.encoder.layer.0.crossattention.self.query.weight', 'roberta.encoder.layer.0.crossattention.self.query.bias', 'roberta.encoder.layer.0.crossattention.self.key.weight', 'roberta.encoder.layer.0.crossattention.self.key.bias', 'roberta.encoder.layer.0.crossattention.self.value.weight', 'roberta.encoder.layer.0.crossattention.self.value.bias', 'roberta.encoder.layer.0.crossattention.output.dense.weight', 'roberta.encoder.layer.0.crossattention.output.dense.bias', 'roberta.encoder.layer.0.crossattention.output.LayerNorm.weight', 'roberta.encoder.layer.0.crossattention.output.LayerNorm.bias', 'roberta.encoder.layer.1.crossattention.self.query.weight', 'roberta.encoder.layer.1.crossattention.self.query.bias', 'roberta.encoder.layer.1.crossattention.self.key.weight', 'roberta.encoder.layer.1.crossattention.self.key.bias', 'roberta.encoder.layer.1.crossatte

In [ ]:
# set special tokens
roberta_shared.config.decoder_start_token_id = tokenizer.bos_token_id                                             
roberta_shared.config.eos_token_id = tokenizer.eos_token_id
# roberta_shared.config.pad_token_id = tokenizer.eos_token_id

# sensible parameters for beam search
# set decoding params                               
roberta_shared.config.max_length = 64
roberta_shared.config.early_stopping = True
roberta_shared.config.no_repeat_ngram_size = 3
roberta_shared.config.length_penalty = 2.0
roberta_shared.config.num_beams = 4
roberta_shared.config.vocab_size = roberta_shared.config.encoder.vocab_size  

In [ ]:
roberta_shared.config

In [ ]:
# !rm seq2seq_trainer.py
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/legacy/seq2seq/seq2seq_trainer.py

!pip install git-python==1.0.3
!pip install sacrebleu==1.4.12
!pip install rouge_score

from seq2seq_trainer import Seq2SeqTrainer
from transformers import TrainingArguments
from dataclasses import dataclass, field
from typing import Optional

clear_output()

In [ ]:
@dataclass
class Seq2SeqTrainingArguments(TrainingArguments):
    label_smoothing: Optional[float] = field(
        default=0.0, metadata={"help": "The label smoothing epsilon to apply (if not zero)."}
    )
    sortish_sampler: bool = field(default=False, metadata={"help": "Whether to SortishSamler or not."})
    predict_with_generate: bool = field(
        default=False, metadata={"help": "Whether to use generate to calculate generative metrics (ROUGE, BLEU)."}
    )
    adafactor: bool = field(default=False, metadata={"help": "whether to use adafactor"})
    encoder_layerdrop: Optional[float] = field(
        default=None, metadata={"help": "Encoder layer dropout probability. Goes into model.config."}
    )
    decoder_layerdrop: Optional[float] = field(
        default=None, metadata={"help": "Decoder layer dropout probability. Goes into model.config."}
    )
    dropout: Optional[float] = field(default=None, metadata={"help": "Dropout probability. Goes into model.config."})
    attention_dropout: Optional[float] = field(
        default=None, metadata={"help": "Attention dropout probability. Goes into model.config."}
    )
    lr_scheduler: Optional[str] = field(
        default="linear", metadata={"help": f"Which lr scheduler to use."}
    )

In [ ]:
import datasets

In [ ]:
# load rouge for validation
rouge = datasets.load_metric("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

Downloading:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

In [ ]:
OUTPUT_DIR = '/content/training/'

In [ ]:
import os

# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
# set training arguments - these params are not really tuned, feel free to change
training_args = Seq2SeqTrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    # evaluate_during_training=True,
    do_train=True,
    do_eval=True,
    logging_steps=200,  # 2000 for full training
    save_steps=1500,  #  500 for full training
    eval_steps=7500,  # 7500 for full training
    warmup_steps=3000,  # 3000 for full training
    num_train_epochs=10, #no comment for full training
    overwrite_output_dir=True,
    save_total_limit=50,
    # fp16=True, 
)

# roberta_shared.config.vocab_size = roberta_shared.config.encoder.vocab_size

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=roberta_shared,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data_batch,
    eval_dataset=val_data_batch,
)

trainer.train()
!mkdir saved
trainer.save_model('./saved')

The `config.pad_token_id` is `None`. Using `config.eos_token_id` = 2 for padding..


Step,Training Loss


In [ ]:
# import datasets
# from transformers import RobertaTokenizer, EncoderDecoderModel, AutoTokenizer
# from sklearn.model_selection import train_test_split

# tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)

# model = EncoderDecoderModel.from_pretrained("/content/saved")
# model.to("cuda")

# # test_data = datasets.load_dataset("xsum", split="test")

# batch_size = 16  # change to 64 for full evaluation

# # map data correctly
# def generate_summary(batch):
#     # Tokenizer will automatically set [BOS] <text> [EOS]
#     inputs = tokenizer(batch["original"], padding="max_length", truncation=True, max_length=256, return_tensors="pt")
#     input_ids = inputs.input_ids.to("cuda")
#     attention_mask = inputs.attention_mask.to("cuda")

#     outputs = model.generate(input_ids, attention_mask=attention_mask)

#     # all special tokens including will be removed
#     output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

#     batch["pred"] = output_str

#     return batch

# results = test_data.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["original"])

# pred_str = results["pred"]
# label_str = results["summary"]

In [ ]:
rdrsegmenter = VnCoreNLP("./vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx2g') 

In [ ]:
text = '''
 Ngày 4/2, lãnh đạo thị xã Hoàng Mai (Nghệ An) cho biết, cơ quan chức năng thị xã vừa phối hợp với đơn vị chức năng tỉnh cách ly, lấy mẫu xét nghiệm cho 1 nam sinh viên trường Đại học FPT có biểu hiện ho sốt sau khi đi từ Hà Nội về nhà.

Trước đó vào ngày 31/1 nam sinh C.T.Ph. (20 tuổi, trú TX. Hoàng Mai, Nghệ An; Sinh viên trường Đại học FPT cơ sở Mỹ Đình) cùng 1 người bạn đi xe máy từ Hà Nội về quê nhà.

Lúc 22h ngày 31/1 nam sinh này về đến quê nhà nhưng không khai báo y tế. Từ ngày 1-3/2, nam sinh Ph. có đi một số nơi trên địa bàn thị xã và tiếp xúc với khoảng 12 người.


Khoảng 20h ngày 3/2, nam sinh này có dấu hiệu sốt 39 độ, đau đầu, đau rát họng, đau mỏi cơ, khó thở nhẹ. Nam sinh này sau đó được đưa vào Bệnh viện Phong da liễu (TX. Hoàng Mai) để khám và cách ly y tế.

Nhận được tin báo, Trung tâm Kiểm soát bệnh tật tỉnh Nghệ An đã trực tiếp ra lấy mẫu bệnh phẩm để xét nghiệm Covid-19. Dự kiến trong chiều cùng ngày sẽ có kết quả.

Hiện cơ quan chức năng đang rà soát những người tiếp xúc gần với nam sinh Ph. để có phương án theo dõi, cách ly và lấy mẫu xét nghiệm.

Trước đó vào đêm 31/1, một chuyến xe khách chở 22 người từ Hà Nội về Nghệ An trong đó có nhiều sinh viên Đại học FPT đi về tại phường Lê Lợi, TP. Vinh (Nghệ An). Cơ quan chức năng sau đó đã đưa 1 trường hợp F1 trên xe khách này đi cách ly tại Trung tâm Y tế huyện Nam Đàn. Riêng những trường hợp còn lại được theo dõi tại nhà.

Sau khi cách ly, các trường hợp này đã được lấy mẫu xét nghiệm và cho kết quả âm tính với Covid-19.
 '''

text = rdrsegmenter.tokenize(text)
text = ' '.join([' '.join(x) for x in text])
text

'Ngày 4/2 , lãnh_đạo thị_xã Hoàng_Mai ( Nghệ_An ) cho biết , cơ_quan_chức_năng thị_xã vừa phối_hợp với đơn_vị chức_năng tỉnh cách_ly , lấy mẫu xét_nghiệm cho 1 nam sinh_viên trường Đại_học FPT có biểu_hiện ho sốt sau khi đi từ Hà_Nội về nhà . Trước đó vào ngày 31/1 nam_sinh C.T.Ph. ( 20 tuổi , trú TX . Hoàng_Mai , Nghệ_An ; Sinh_viên trường Đại_học FPT cơ_sở Mỹ_Đình ) cùng 1 người bạn đi xe_máy từ Hà_Nội về quê nhà . Lúc 22h ngày 31/1 nam_sinh này về đến quê nhà nhưng không khai_báo y_tế . Từ ngày 1-3/2 , nam_sinh Ph . có đi một_số nơi trên địa_bàn thị_xã và tiếp_xúc với khoảng 12 người . Khoảng 20h ngày 3/2 , nam_sinh này có dấu_hiệu sốt 39 độ , đau_đầu , đau rát họng , đau mỏi cơ , khó thở nhẹ . Nam_sinh này sau đó được đưa vào Bệnh_viện Phong da_liễu ( TX . Hoàng_Mai ) để khám và cách_ly y_tế . Nhận được tin báo , Trung_tâm Kiểm_soát bệnh_tật tỉnh Nghệ_An đã trực_tiếp ra lấy mẫu bệnh_phẩm để xét_nghiệm Covid-19 . Dự_kiến trong chiều cùng ngày sẽ có kết_quả . Hiện cơ_quan_chức_năng đ

In [ ]:
inputs = tokenizer(text, padding="max_length", truncation=True, max_length=256, return_tensors="pt")
input_ids = inputs.input_ids.to("cuda")
attention_mask = inputs.attention_mask.to("cuda")

outputs = model.generate(input_ids, attention_mask=attention_mask)

# all special tokens including will be removed
output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
output_str

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1065: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


['Vũ_Văn_Vũ_Văn_liên_kết liên_kết nâng nâng nâng liên_kết liên_kết liên_kết để liên_kết liên_kết biệt biệt biệt liên_kết để để liên_kết nâng liên_kết nâng để liên_kết để nâng nâng nhau liên_kết liên_kết nhau liên_kết nâng biệt liên_kết liên_kết ing liên_kết liên_kết Vũ_Văn_nâng nâng để để để nâng liên_kết nhau để liên_kết ing ing liên_kết nâng ing ing ing nâng nâng ing nâng liên_kết']

In [ ]:
!pip install gdown

In [ ]:
import gdown

!gdown https://drive.google.com/drive/folders/1uqGM9B8cvthsRP6hIXI7c7XXsz8uNOpk?usp=sharing

/usr/local/lib/python2.7/dist-packages/gdown/parse_url.py:31: UserWarning: You specified Google Drive Link but it is not the correct link to download the file. Maybe you should try: https://drive.google.com/uc?id=None
  .format(url='https://drive.google.com/uc?id={}'.format(file_id))
Downloading...
From: https://drive.google.com/drive/folders/1uqGM9B8cvthsRP6hIXI7c7XXsz8uNOpk?usp=sharing
To: /content/1uqGM9B8cvthsRP6hIXI7c7XXsz8uNOpk?usp=sharing
227kB [00:00, 1.38MB/s]
